In [1]:
import torch
import torchvision
import numpy as np
import os.path as osp
import os
import time


In [2]:
CUDA = torch.cuda.is_available()
CUDA

False

In [3]:
# GLOBALS
# vot_path = "../../../vot2016/"
vot_path = "/home/zabulskyy/Datasets/vot2016/"
# yolo_pred_path = "../yolo_predictions/extended"
yolo_pred_path = "/mnt/sshfs/YOLO-tracker/yolo_predictions/extended"
crit_vals = {
    "iou": 1,
    "cc": 1,               # class_correspondence
    "id": 1,               # initial dist 
    "vd": 1                # vector dist
#     "dist2": 1,
}
with open("../../yoloTorch/data/coco.names", "r") as f:
    classes = f.read().split("\n")[:-1]
print(classes)
dist = torch.nn.modules.PairwiseDistance()
vot_classes = [x for x in os.listdir(vot_path) if not x.endswith(".txt")]
print(vot_classes)

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']
['blanket', 'bolt2', 'traffic', 'ball1', 'leaves', 'pedestrian1', 'singer3', 'helicopter', 'godfather', 'soccer1', 'book', 'tunnel', '

In [4]:
def read_csv(file="correlations.csv"):
    with open(file, 'r') as file:
        l = file.read().split("\n")
        l.pop(-1)
        l = [[float(x) for x in y.split(",")[:-1]] for y in l]   
        return torch.tensor(l)
    

In [5]:
cm = read_csv()

In [6]:
def dist2(row1, row2):
    pass

def the_closest(row_to_compare, rows):
    pass

def iou(boxA, boxB):
    pass

def the_most_iou(row_to_compare, rows):
    pass

def interpolate(data):
    pass

def read_gt(vot_class, vot_path=vot_path, force_square=True):
    pass

def read_yolo_pred(vot_class, yolo_pred_path=yolo_pred_path, force_square=True):
    pass

def eval_energy(bb1, bb2, cv=crit_vals):
    pass

def class_correlations(class_probs1, class_probs2):
    pass

def eval_class_corr(cls1, cls2, correlations=cm):
    pass

In [7]:
def dist2(row1, row2):
    x11, y11, x12, y12 = row1[1:5] if len(row1) != 4 else row1[:]
    x21, y21, x22, y22 = row2[1:5] if len(row2) != 4 else row2[:]
    x1, y1 = (x11 + x12) / 2, (y11 + y12) / 2
    x2, y2 = (x21 + x22) / 2, (y21 + y22) / 2
    X = (x1 - x2) ** 2
    Y = (y1 - y2) ** 2
    res = float((X + Y) ** 0.5)
    return res

def the_closest(row_to_compare, rows):
    m, i = np.infty, 0
    for n, row in enumerate(rows):
        d = dist2(row, row_to_compare)
        if d < m:
            m = float(d)
            i = n
    return rows[i], i

In [8]:
def iou(boxA, boxB):
    boxA = boxA[1:5] if len(boxA) != 4 else boxA
    boxB = boxB[1:5] if len(boxB) != 4 else boxB

    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    res = interArea / float(boxAArea + boxBArea - interArea)
    return res

def the_most_iou(row_to_compare, rows):
    M, i = 0, 0
    for n, row in enumerate(rows):
        v = iou(row, row_to_compare)
        if v > M:
            M = float(v)
            i = n
    return rows[i], i

In [9]:
def generate_subframes(row1, row2, width):
    frame1, frame2 = int(row1[0]), int(row2[0])
    diff = frame1 - frame2
    if diff == 1:
        return None
    result = torch.zeros((frame2 - frame1 - 1, width))
    # result[:, -1] = row1[-1]
    step = (row1 - row2) / diff
    for frame_n in range(frame2 - frame1 - 1):
        result[frame_n] = row1 + (step) * (frame_n + 1)
    return result

In [10]:
def merge_frames(*rows):
    result = torch.zeros((1, 8))
    n = len(rows)
    for row in rows:
        result += row
    return result / n

In [11]:
def fill_first(tensor):
    FIRST = tensor[0]
    for i in range(int(FIRST[0]) - 1, -1, -1):
        row = FIRST
        row[0] = i
        row = row.view((1, -1))
        tensor = torch.cat((row, tensor))
    return tensor

In [12]:
def fill_last(tensor, NUM_FRAMES):
    last = tensor[tensor[:, 0] != 0][-1]
    for i in range(int(last[0]) + 1, NUM_FRAMES):
        row = last
        row[0] = i
        row = row.view((1, -1))
        tensor[i] = row
    return tensor

In [13]:
def most_frequent_class(results):
    counts = np.bincount(results[:, -1])
    return np.argmax(counts)

In [14]:
def interpolate(data):
    """
    interpolate gaps
    choose one detection among multiple, by picking the closest one by the euclidean distance
    ignore class labels
    :param OUTPUT: tensor
    :param NUM_FRAMES:
    :param CUDA: bool - if cuda is available
    :return: interpolated tensor
    """
    OUTPUT = data["output"]
    NUM_FRAMES = data["num_frames"]
    CUDA = data["CUDA"]

    width = OUTPUT.size()[1]
    result = torch.zeros((NUM_FRAMES, width))
    if CUDA:
        result = result.cuda()

    if OUTPUT[0][0] != 0:
        OUTPUT = fill_first(OUTPUT)

    output_iter = 0
    res_iter = 0
    while res_iter != NUM_FRAMES:
        if (output_iter > len(OUTPUT) - 1):
            # break
            result = fill_last(result, NUM_FRAMES)
            break
        if output_iter == OUTPUT.shape[0] - 1:
            result[res_iter] = OUTPUT[output_iter]
            output_iter += 1
            res_iter += 1
            continue

        # next frame is interpolated
        if OUTPUT[output_iter][0] == OUTPUT[output_iter + 1][0] - 1:
            result[res_iter] = OUTPUT[output_iter]
            output_iter += 1
            res_iter += 1
            continue

        # missing detection on some frames, fill with missing
        if OUTPUT[output_iter][0] + 1 < OUTPUT[output_iter + 1][0]:
            subframes = generate_subframes(
                OUTPUT[output_iter], OUTPUT[output_iter + 1], width)
            result[res_iter] = OUTPUT[output_iter]
            for subframe in subframes:
                res_iter += 1
                result[res_iter] = subframe
            output_iter += 1
            res_iter += 1
            continue

        # this frame contains multiple detections, have to choose one
        if OUTPUT[output_iter][0] == OUTPUT[output_iter + 1][0]:
            frame = OUTPUT[output_iter][0]
            to_cut = OUTPUT[OUTPUT[:, 0] == frame]
            # closest, _ = the_closest(result[res_iter - 1], to_cut)
            closest, _ = the_most_iou(result[res_iter - 1], to_cut)
#             closest, _ = the_closest_vec(result[res_iter - 1], to_cut)
            result[res_iter] = closest

            output_iter += len(to_cut)
            res_iter += 1

            if output_iter >= OUTPUT.shape[0] - 1:
                break

            # missing detection on some frames, fill with missing
            if OUTPUT[output_iter - 1][0] + 1 < OUTPUT[output_iter][0]:
                subframes = generate_subframes(
                    result[res_iter - 1], OUTPUT[output_iter], width)
                # result[res_iter] = OUTPUT[output_iter]
                for subframe in subframes:
                    result[res_iter] = subframe
                    res_iter += 1
            continue
    return result

In [15]:
def read_gt(vot_class, vot_path=vot_path, force_square=True):
    gt_file = osp.join(vot_path, vot_class, "groundtruth.txt")
    with open(gt_file, 'r') as file:
        l =file.read().split("\n")
        l.pop(-1)
        l = [[float(x) for x in y.split(",")] for y in l]
        if force_square:
            l = [[min(m[::2]), min(m[1::2]), max(m[::2]), max(m[1::2])] for m in l]   
        return torch.tensor(l)
    
def read_full_gt(vot_path=vot_path, force_square=True):
    vot_classes = [x for x in os.listdir(vot_path) if not x.endswith(".txt")]
    X = list()
    for vot_class in vot_classes:
        gt_file = osp.join(vot_path, vot_class, "groundtruth.txt")
        with open(gt_file, 'r') as file:
            l =file.read().split("\n")
            l.pop(-1)
            l = [[float(x) for x in y.split(",")] for y in l]
            if force_square:
                l = [[min(m[::2]), min(m[1::2]), max(m[::2]), max(m[1::2])] for m in l]   
        X.append(l)
    return torch.tensor(X)

In [16]:
def read_yolo_pred(vot_class, yolo_pred_path=yolo_pred_path, pred_format="csv", first_len=True):
    yolo_pred_file = osp.join(yolo_pred_path, vot_class + "." + pred_format)
    with open(yolo_pred_file, 'r') as file:
        l =file.read().split("\n")
        l.pop(-1)
        im_len = -1
        if first_len:
            im_len = float(l.pop(0))
        l = [[float(x) for x in y.split(",")] for y in l]            
        return torch.tensor(l), im_len
    

In [17]:
yolo_pred = dict()

for i in vot_classes:
    print(i)
    yolo_pred[i] = read_yolo_pred(i)
    

blanket
bolt2
traffic
ball1
leaves
pedestrian1
singer3
helicopter
godfather
soccer1
book
tunnel
gymnastics1
basketball
birds1
car1
ball2
birds2
motocross1
bmx
gymnastics2
fish3
motocross2
soccer2
octopus
fernando
shaking
crossing
singer2
soldier
pedestrian2
nature
butterfly
road
iceskater2
sphere
fish2
graduate
marching
rabbit
car2
matrix
girl
tiger
gymnastics4
wiper
racing
dinosaur
bolt1
hand
handball1
bag
sheep
iceskater1
fish1
singer1
gymnastics3
fish4
glove
handball2


In [18]:
yolo_pred.keys()

dict_keys(['blanket', 'bolt2', 'traffic', 'ball1', 'leaves', 'pedestrian1', 'singer3', 'helicopter', 'godfather', 'soccer1', 'book', 'tunnel', 'gymnastics1', 'basketball', 'birds1', 'car1', 'ball2', 'birds2', 'motocross1', 'bmx', 'gymnastics2', 'fish3', 'motocross2', 'soccer2', 'octopus', 'fernando', 'shaking', 'crossing', 'singer2', 'soldier', 'pedestrian2', 'nature', 'butterfly', 'road', 'iceskater2', 'sphere', 'fish2', 'graduate', 'marching', 'rabbit', 'car2', 'matrix', 'girl', 'tiger', 'gymnastics4', 'wiper', 'racing', 'dinosaur', 'bolt1', 'hand', 'handball1', 'bag', 'sheep', 'iceskater1', 'fish1', 'singer1', 'gymnastics3', 'fish4', 'glove', 'handball2'])

In [19]:
# compute class correlation
def ccc(classes_probs):
    # classes_probs.shape = (bboxes, classes)
    corr_mat = torch.zeros((classes_probs.shape[1], classes_probs.shape[1], ))

    for cls in range(len(classes_probs[0])):
        col = classes_probs[:, cls].view((-1, 1))
        tX = col * classes_probs
        sX = torch.sum(tX, dim=0)
        corr_mat[cls] = sX
    return corr_mat

def corr(col1, col2):
    mean1, mean2 = torch.mean(col1), torch.mean(col2)
    cm1, cm2 = col1 - mean1, col2 - mean2
    top = sum(cm1 * cm2)
    bot = torch.sqrt(sum(cm1 ** 2) * sum(cm2 ** 2))
    return top / bot

# compute class correlation coefficient
def cccc(classes_probs):
    # classes_probs.shape = (bboxes, classes)
    corr_mat = torch.zeros((classes_probs.shape[1], classes_probs.shape[1], ))
    num_classes = len(classes_probs[0])
    for cls1 in range(num_classes):
        col1 = classes_probs[:, cls1].view((-1, 1))
        for cls2 in range(cls1 + 1, num_classes):
            print("{}.{} / {}".format(cls1+1, cls2+1, num_classes))
            col2 = classes_probs[:, cls2].view((-1, 1))
            _corr = corr(col1, col2)
#             _corr = np.corrcoef(col1, col2)
            corr_mat[cls1, cls2] = _corr
    return corr_mat


In [20]:
def read_all_yolo_preds(path=yolo_pred_path, pred_format="csv"):
    X = []
    for vot_class in os.listdir(path):
        yolo_pred_file = osp.join(path, vot_class)
        with open(yolo_pred_file, 'r') as file:
            l = file.read().split("\n")
            l.pop(-1)
            l.pop(0)
            im_len = -1
            for i in l:
                X.append([float(x) for x in i.split(",")])
    X = torch.tensor(X)
#     cls_probs = X[:, 6:]
    return X

In [21]:
def eval_single_class_corr(cls1 : int, cls2 : int, correlations=cm):
    return float(cm[cls1][cls2])

def eval_whole_class_corr(cls1, cls2, correlations=cm):
    # TODO: consider if this worth it
    # too complicated yet
    pass

In [22]:
def normalise(tensor):
    mean = torch.mean(tensor)
    std = torch.std(tensor)
    return (tensor - mean) / std

In [23]:
def write_csv(matrix, file="correlations.csv"):
    for i in matrix.tolist():
        for j in i:
            print(j, sep=", ", end=", ", file=open(file, "a+"))
        print(file=open(file, "a+"))

In [24]:
def eval_energy(bb1, bb2, cv=crit_vals, ban_negative_cc=True):
    box1, box2 = bb1[1:5], bb2[1:5]
    cls1, cls2 = bb1[6: ], bb2[6: ]  # 5'th column is the precision of the bbox
    class1, class2 = torch.argmax(cls1), torch.argmax(cls2)
    
#     _dist2 = dist2(box1, box2)  # probably not
    
    _iou = iou(box1, box2)
    
    _cc = eval_single_class_corr(class1, class2)
    
    
    _vd = vec_dist(bb1, bb2)
    
    
    return {"iou": _iou, "cc": _cc, "vd": 1 / _vd}

In [25]:
def cebtf(frame1, frame2, cv=crit_vals, ban_negative_cc=True):  # calc_energies_between_two_frames
    _boxes = torch.zeros((len(frame1)))
    _energies = torch.zeros((len(frame1)))
    for i in range(len(frame1)):
        max_energy = 0
        max_energy_index = 0
        
        
        ious = torch.zeros(len(frame2))
        
        ccs = torch.zeros(len(frame2))
        
        vds = torch.zeros(len(frame2))
        
        for j in range(len(frame2)):
            energy = eval_energy(frame1[i], frame2[j], cv=crit_vals, ban_negative_cc=ban_negative_cc)
            
            if ban_negative_cc and energy["cc"] < 0:        
                pass  # values are already 0
            else:          
                ious[j] = energy["iou"] * crit_vals["iou"]
                ccs[j] = energy["cc"] * crit_vals["cc"]
                vds[j] = energy["vd"] * crit_vals["vd"]
                
        # normalize
        ious = ious / (torch.max(ious) + 1e-10)
        ccs = ccs / (torch.max(ccs) + 1e-10)
        vds = vds / (torch.max(vds) + 1e-10)
        
        energies = sum((ious, ccs, vds))

        _boxes[i] = torch.argmax(energies)
        _energies[i] = torch.max(energies)
    return _boxes, _energies
          

In [85]:
boxes, energy = cebtf(a, b)

iou 0.0014421939849853516
ccs 6.461143493652344e-05
vds 0.0032951831817626953
iou 0.000301361083984375
ccs 4.124641418457031e-05
vds 0.0017719268798828125
iou 0.00029778480529785156
ccs 3.314018249511719e-05
vds 0.001020669937133789
iou 0.0001666545867919922
ccs 0.0003628730773925781
vds 0.0005123615264892578
iou 0.000141143798828125
ccs 0.0003104209899902344
vds 0.0005028247833251953
iou 0.00027561187744140625
ccs 0.0002601146697998047
vds 0.0006182193756103516
iou 0.00011897087097167969
ccs 0.00025272369384765625
vds 0.00040531158447265625
iou 0.00010848045349121094
ccs 0.00024628639221191406
vds 0.0003981590270996094
iou 0.0001227855682373047
ccs 0.00022745132446289062
vds 0.00034356117248535156
iou 9.489059448242188e-05
ccs 0.00022077560424804688
vds 0.0007102489471435547
iou 0.00010800361633300781
ccs 1.1205673217773438e-05
vds 0.00011610984802246094
iou 0.0008864402770996094
ccs 0.00016307830810546875
vds 0.0005381107330322266
iou 0.00014448165893554688
ccs 9.226799011230469e-05


iou 0.0002377033233642578
ccs 2.0742416381835938e-05
vds 0.0004954338073730469
iou 0.00012803077697753906
ccs 1.9550323486328125e-05
vds 0.0005331039428710938
iou 8.320808410644531e-05
ccs 8.845329284667969e-05
vds 0.00035643577575683594
iou 0.0006072521209716797
ccs 9.441375732421875e-05
vds 0.00037097930908203125
iou 0.0006494522094726562
ccs 9.322166442871094e-05
vds 0.0004715919494628906
iou 0.0005578994750976562
ccs 1.811981201171875e-05
vds 0.0004563331604003906
iou 0.0006458759307861328
ccs 8.606910705566406e-05
vds 0.00033354759216308594
iou 0.000492095947265625
ccs 7.724761962890625e-05
vds 0.00033211708068847656
iou 7.557868957519531e-05
ccs 8.153915405273438e-05
vds 0.0003349781036376953
iou 0.00041031837463378906
ccs 1.5020370483398438e-05
vds 0.0005023479461669922
iou 8.296966552734375e-05
ccs 1.9311904907226562e-05
vds 0.00031685829162597656
iou 8.654594421386719e-05
ccs 0.00010132789611816406
vds 0.00038909912109375
iou 7.653236389160156e-05
ccs 0.00017905235290527344
vd

vds 0.0004780292510986328
iou 0.0002899169921875
ccs 2.1219253540039062e-05
vds 0.00054168701171875
iou 0.00013256072998046875
ccs 1.7881393432617188e-05
vds 0.0005557537078857422
iou 8.153915405273438e-05
ccs 0.00018024444580078125
vds 0.00028228759765625
iou 7.534027099609375e-05
ccs 0.0001761913299560547
vds 0.00030422210693359375
iou 7.534027099609375e-05
ccs 0.00017762184143066406
vds 0.0002799034118652344
iou 7.581710815429688e-05
ccs 0.0001785755157470703
vds 0.00028324127197265625
iou 7.557868957519531e-05
ccs 0.0001804828643798828
vds 0.00028204917907714844
iou 7.462501525878906e-05
ccs 0.00017762184143066406
vds 0.0002753734588623047
iou 7.462501525878906e-05
ccs 0.00018858909606933594
vds 0.0002753734588623047
iou 7.367134094238281e-05
ccs 0.000179290771484375
vds 0.0002856254577636719
iou 8.606910705566406e-05
ccs 6.437301635742188e-05
vds 0.0004420280456542969
iou 7.677078247070312e-05
ccs 0.00019407272338867188
vds 0.0002765655517578125
iou 7.534027099609375e-05
ccs 0.000

In [26]:
def the_most_iou(row_to_compare, rows):
    M, i = 0, 0
    for n, row in enumerate(rows):
        v = iou(row, row_to_compare)
        if v > M:
            M = float(v)
            i = n
    return rows[i], i

def pp_first(data):
    # detects the closest object to FIRST gt box and removes the rest on the FIRST frame
    CUDA = data["CUDA"]
    output = data["output"]
    first = data["first"]

    tcc, _ = the_most_iou(first, output[:, 1:5])
#     print("PP_FIRST")
#     print(first, output, tcc, sep="\n")
#     print("PP_FIRST")
    
    # only detections close to the true FIRST frame
    
    output = replace_first_frame(tcc, output)
    return output

def replace_first_frame(to_replace, OUTPUT):
    to_replace = to_replace[:4]
#     print(to_replace)
    OUTPUT = OUTPUT[OUTPUT[:, 0] != 0]
#     print(OUTPUT.shape)
    to_replace = to_replace.view((1, -1))
    _to_replace = torch.zeros((1, 8))
    _to_replace[0][1:5] = to_replace[0]
    return torch.cat((_to_replace, OUTPUT))

In [27]:
def get_first(vot_class):
    with open(osp.join(vot_path, vot_class, "groundtruth.txt"), 'r') as file:
        l = [float(x) for x in file.read().split("\n")[0].split(",")]
        X, Y = l[::2], l[1::2]
        l = [min(X), min(Y), max(X), max(Y)]
        l = torch.tensor([float(x) for x in l])
        return l

In [28]:
def initial_energy(first_boxes, method="ltd2", cv=crit_vals, ): 
    l = first_boxes.shape[0]
    ies = torch.zeros(l)  # initial_energies
    # ltd2 = least total dist2
    for i in range(l):
        for j in range(l):
            ie = dist2(first_boxes[i],first_boxes[j])
            ies[i] += ie
    ies = 1 / ies * crit_vals["id"]
    return ies
            

In [29]:
def vec_dist(vec1, vec2, dist=dist):
    
    vd = None
    
    if len(vec1) > 80:
        vec1[torch.argmax(vec1[6:]) + 6] = 1
        vec2[torch.argmax(vec2[6:]) + 6] = 1
        vd = dist(vec1[6:].view([1, -1]), vec2[6:].view([1, -1]))
    else:
        vec1[torch.argmax(vec1)] = 1
        vec2[torch.argmax(vec2)] = 1
        vd = dist(vec1.view([1, -1]), vec2.view([1, -1])) 
    return vd

def the_closest_vec(row_to_compare, rows):
    m, i = np.infty, 0
    for n, row in enumerate(rows):
        d = vec_dist(row, row_to_compare)
        if d < m:
            m = float(d)
            i = n
    return rows[i], i
    

In [30]:
def do_dp(vot_class, total_res):
    yp, imlen = yolo_pred[vot_class]  # yolo predictions
    
    data = dict()
    data["output"] = yp
    data["num_frames"] = int(imlen)
    data["CUDA"] = CUDA
    data["first"] = get_first(vot_class)
    
    if CUDA:
        data["output"] = data["output"].cuda()
        data["first"] = data["first"].cuda()
        
    
#     print(data["first"])
#     print(data["output"][:10])
#     data["output"] = pp_first(data)
#     print(data["output"][:4])

    
    frames_with_detection = sorted(list(set(data["output"][:,0].tolist())))
    ffwdi = frames_with_detection[0]  # first_frame_with_detection_index
    first_boxes = data["output"][data["output"][:, 0] == ffwdi]
    
    path = torch.zeros((len(first_boxes), imlen))
    energies = torch.zeros((len(first_boxes), imlen))
    
    prev_boxes = first_boxes
    path[:, 0] = first_boxes[0][0]
    
    
    # calculate energies
#     print("energies")
    for n, i in enumerate(frames_with_detection[:]):
        
#         if n == 0:
#             continue
            
#         if n % 100 == 99:
#             print(str(n / len(frames_with_detection[1:]) * 100)[:4], end="% ")
    
        i_boxes = data["output"][data["output"][:, 0] == i]
        
        boxes, energy = cebtf(prev_boxes, i_boxes)
        
        path[:, int(n)] = boxes
        energies[:, int(n)] += energy
        
        prev_boxes = i_boxes[boxes.long()]
        
    verdict = torch.sum(energies, dim=1)
    
#     init energies   
    ie = initial_energy(first_boxes, method="ltd2")
    ie = ie / max(ie) * max(verdict) * crit_vals["id"]
    
    if verdict.shape != torch.Size([1]):
        verdict += ie

    
    first_bb_candidates = (verdict == torch.max(verdict)).nonzero().view(-1).tolist()
    res_i = path[first_bb_candidates].view(-1).int()
    res = torch.zeros((len(frames_with_detection), 5))

    # prepare for interpolation
    for n, i in enumerate(frames_with_detection[:]):
        cfp = data["output"][data["output"][:, 0] == i]  # current_frame_predictions
        index = res_i[n]
        cp = cfp[index] # chosen_prediction
        res[int(n)] = cp[:5]
        
    
#     print("\ninterpolation")

    total_res[vot_class] = interpolate(data)[:, 1:5].tolist()
    
    return total_res
    

In [31]:
total_res=dict()
CUDA = False

In [107]:
for vot_class in ["birds2"]:
    do_dp(vot_class, total_res)
    
    

read predictions 1.6689300537109375e-06
0'th frame'
iou 0.00017595291137695312
ccs 2.86102294921875e-05
vds 0.0007457733154296875
iou 0.00013136863708496094
ccs 8.058547973632812e-05
vds 0.0006110668182373047
iou 0.0001010894775390625
ccs 0.00024271011352539062
vds 0.00038909912109375
iou 9.632110595703125e-05
ccs 6.651878356933594e-05
vds 0.0005233287811279297
iou 0.00010204315185546875
ccs 0.0002110004425048828
vds 0.0003330707550048828
iou 9.822845458984375e-05
ccs 6.818771362304688e-05
vds 0.00048661231994628906
iou 9.083747863769531e-05
ccs 6.651878356933594e-05
vds 0.0004649162292480469
iou 8.940696716308594e-05
ccs 6.67572021484375e-05
vds 0.0004658699035644531
iou 0.00014925003051757812
ccs 2.288818359375e-05
vds 0.0007066726684570312
iou 0.000110626220703125
ccs 7.009506225585938e-05
vds 0.0006604194641113281
iou 0.00021004676818847656
ccs 9.274482727050781e-05
vds 0.00038886070251464844
iou 9.489059448242188e-05
ccs 2.3365020751953125e-05
vds 0.001087188720703125
iou 0.000106

vds 0.0005598068237304688
iou 0.0003204345703125
ccs 1.6450881958007812e-05
vds 0.0005567073822021484
iou 8.296966552734375e-05
ccs 1.5974044799804688e-05
vds 0.0006203651428222656
iou 8.392333984375e-05
ccs 1.7881393432617188e-05
vds 0.0004947185516357422
iou 0.00016927719116210938
ccs 1.6689300537109375e-05
vds 0.0005490779876708984
iou 8.630752563476562e-05
ccs 1.6689300537109375e-05
vds 0.0004315376281738281
iou 0.00036978721618652344
ccs 0.00018262863159179688
vds 0.0002696514129638672
iou 7.700920104980469e-05
ccs 0.0001900196075439453
vds 0.00024366378784179688
iou 7.414817810058594e-05
ccs 0.00017762184143066406
vds 0.00025153160095214844
iou 7.486343383789062e-05
ccs 0.00017571449279785156
vds 0.00025153160095214844
iou 7.462501525878906e-05
ccs 0.00017595291137695312
vds 0.00025916099548339844
iou 7.343292236328125e-05
ccs 1.3589859008789062e-05
vds 0.0004181861877441406
iou 7.367134094238281e-05
ccs 0.0008537769317626953
vds 0.0003898143768310547
iou 8.034706115722656e-05
cc

ccs 0.00022268295288085938
vds 0.0003330707550048828
iou 0.0004949569702148438
ccs 0.00018024444580078125
vds 0.0002586841583251953
iou 7.414817810058594e-05
ccs 0.00017762184143066406
vds 0.00030112266540527344
iou 8.535385131835938e-05
ccs 0.0001773834228515625
vds 0.0002465248107910156
iou 8.845329284667969e-05
ccs 0.0001838207244873047
vds 0.0002601146697998047
iou 7.653236389160156e-05
ccs 0.0001747608184814453
vds 0.0002491474151611328
iou 7.486343383789062e-05
ccs 0.00017499923706054688
vds 0.0002510547637939453
iou 8.749961853027344e-05
ccs 0.00017547607421875
vds 0.00025153160095214844
iou 9.298324584960938e-05
ccs 1.9311904907226562e-05
vds 0.0004277229309082031
iou 7.653236389160156e-05
ccs 0.00017786026000976562
vds 0.0002522468566894531
iou 7.486343383789062e-05
ccs 9.918212890625e-05
vds 0.0003390312194824219
iou 7.510185241699219e-05
ccs 0.0001900196075439453
vds 0.0002319812774658203
iou 6.437301635742188e-05
ccs 0.00015497207641601562
vds 0.00022363662719726562
iou 6.5

ccs 0.00041604042053222656
vds 0.0006406307220458984
iou 7.987022399902344e-05
ccs 0.00017714500427246094
vds 0.0002586841583251953
iou 0.00010728836059570312
ccs 6.29425048828125e-05
vds 0.0004267692565917969
iou 7.581710815429688e-05
ccs 0.00017118453979492188
vds 0.00024700164794921875
iou 7.677078247070312e-05
ccs 0.00017976760864257812
vds 0.00024580955505371094
iou 7.224082946777344e-05
ccs 0.00016760826110839844
vds 0.00025153160095214844
iou 8.368492126464844e-05
ccs 5.412101745605469e-05
vds 0.0003826618194580078
iou 7.390975952148438e-05
ccs 9.632110595703125e-05
vds 0.00032520294189453125
iou 7.176399230957031e-05
ccs 9.5367431640625e-05
vds 0.0003249645233154297
iou 7.176399230957031e-05
ccs 0.00016760826110839844
vds 0.000240325927734375
iou 7.128715515136719e-05
ccs 0.0001685619354248047
vds 0.0002448558807373047
iou 7.939338684082031e-05
ccs 1.1682510375976562e-05
vds 0.0003972053527832031
iou 7.152557373046875e-05
ccs 0.00017261505126953125
vds 0.00024366378784179688
io

iou 0.000518798828125
ccs 5.9604644775390625e-06
vds 0.0003952980041503906
iou 0.0007119178771972656
ccs 0.0001575946807861328
vds 0.0003974437713623047
iou 0.000118255615234375
ccs 2.0503997802734375e-05
vds 0.000507354736328125
iou 7.05718994140625e-05
ccs 0.00025653839111328125
vds 0.00035572052001953125
iou 0.0002460479736328125
ccs 3.5762786865234375e-05
vds 0.0003037452697753906
iou 7.43865966796875e-05
ccs 0.00015401840209960938
vds 0.00023698806762695312
iou 9.822845458984375e-05
ccs 1.6927719116210938e-05
vds 0.00037789344787597656
iou 6.771087646484375e-05
ccs 0.0001697540283203125
vds 0.00024580955505371094
iou 6.556510925292969e-05
ccs 0.00016832351684570312
vds 0.00021910667419433594
iou 6.532669067382812e-05
ccs 0.00015234947204589844
vds 0.000217437744140625
iou 6.556510925292969e-05
ccs 0.0001537799835205078
vds 0.00016832351684570312
iou 6.580352783203125e-05
ccs 1.5020370483398438e-05
vds 0.000370025634765625
iou 6.604194641113281e-05
ccs 0.00015306472778320312
vds 0.

...

vds 0.0008378028869628906
iou 0.0001964569091796875
ccs 0.0003809928894042969
vds 0.00042057037353515625
iou 0.00011992454528808594
ccs 0.00019407272338867188
vds 0.0006899833679199219
iou 0.0001461505889892578
ccs 0.00018310546875
vds 0.0005753040313720703
iou 0.000125885009765625
ccs 0.00016641616821289062
vds 0.0006055831909179688
iou 0.00013303756713867188
ccs 0.0002961158752441406
vds 0.0003390312194824219
iou 0.0001316070556640625
ccs 2.09808349609375e-05
vds 0.0006902217864990234
iou 0.00012350082397460938
ccs 0.00029206275939941406
vds 0.0006918907165527344
iou 0.00014138221740722656
ccs 2.288818359375e-05
vds 0.0008187294006347656
iou 0.0001232624053955078
ccs 0.0002894401550292969
vds 0.0004661083221435547
iou 0.00015020370483398438
ccs 0.00029087066650390625
vds 0.00043463706970214844
iou 0.00024127960205078125
ccs 0.00029468536376953125
vds 0.0004737377166748047
iou 0.00012111663818359375
ccs 0.00020122528076171875
vds 0.0007658004760742188
iou 0.0001442432403564453
ccs 3.3

vds 0.0008120536804199219
iou 0.00014710426330566406
ccs 0.0001804828643798828
vds 0.0006909370422363281
iou 0.00013828277587890625
ccs 0.00010800361633300781
vds 0.0007674694061279297
iou 0.00013208389282226562
ccs 0.0003483295440673828
vds 0.0005445480346679688
iou 0.0001468658447265625
ccs 0.0003070831298828125
vds 0.0004901885986328125
iou 0.000133514404296875
ccs 0.0003380775451660156
vds 0.00046443939208984375
iou 0.00018835067749023438
ccs 0.00017690658569335938
vds 0.0006458759307861328
iou 0.00019311904907226562
ccs 0.0001461505889892578
vds 0.0006730556488037109
iou 0.00018405914306640625
ccs 0.00010037422180175781
vds 0.0006754398345947266
iou 0.00013303756713867188
ccs 0.00030612945556640625
vds 0.00044417381286621094
iou 0.00012683868408203125
ccs 0.00030303001403808594
vds 0.0004315376281738281
iou 0.00018310546875
ccs 0.00033473968505859375
vds 0.0004837512969970703
iou 0.00018477439880371094
ccs 0.00035858154296875
vds 0.00045871734619140625
iou 0.00013017654418945312
c

vds 0.0009474754333496094
iou 0.00022411346435546875
ccs 2.7894973754882812e-05
vds 0.0009469985961914062
iou 0.0001430511474609375
ccs 2.8848648071289062e-05
vds 0.0009641647338867188
iou 0.0001556873321533203
ccs 3.0994415283203125e-05
vds 0.0012784004211425781
iou 0.0003483295440673828
ccs 0.0001881122589111328
vds 0.0006108283996582031
iou 0.0017876625061035156
ccs 2.5510787963867188e-05
vds 0.0007886886596679688
iou 0.0004980564117431641
ccs 0.00010085105895996094
vds 0.0006618499755859375
cebtf 0.638803243637085
76'th frame'
iou 0.000141143798828125
ccs 0.00031495094299316406
vds 0.000522613525390625
iou 0.0001366138458251953
ccs 3.0040740966796875e-05
vds 0.0007784366607666016
iou 0.00013017654418945312
ccs 0.00011301040649414062
vds 0.00086212158203125
iou 0.00013375282287597656
ccs 3.24249267578125e-05
vds 0.0008351802825927734
iou 0.00012969970703125
ccs 0.0003097057342529297
vds 0.0004425048828125
iou 0.00012826919555664062
ccs 0.0002276897430419922
vds 0.0006880760192871094

vds 0.0007939338684082031
iou 0.0003771781921386719
ccs 0.00031447410583496094
vds 0.00046539306640625
iou 0.0001285076141357422
ccs 3.8623809814453125e-05
vds 0.0008637905120849609
iou 0.00015163421630859375
ccs 0.0001742839813232422
vds 0.0007424354553222656
iou 0.00014472007751464844
ccs 0.0003330707550048828
vds 0.00047588348388671875
iou 0.0002319812774658203
ccs 3.3855438232421875e-05
vds 0.0008864402770996094
iou 0.0002353191375732422
ccs 0.00012993812561035156
vds 0.001085042953491211
iou 0.0002448558807373047
ccs 0.00023794174194335938
vds 0.0006859302520751953
iou 0.00022745132446289062
ccs 2.5272369384765625e-05
vds 0.0008697509765625
iou 0.0002548694610595703
ccs 3.790855407714844e-05
vds 0.0007061958312988281
iou 0.0007929801940917969
ccs 2.0503997802734375e-05
vds 0.0011510848999023438
iou 0.00017571449279785156
ccs 0.00011491775512695312
vds 0.0006809234619140625
iou 0.00014901161193847656
ccs 0.00033783912658691406
vds 0.0003781318664550781
iou 0.00012683868408203125
cc

KeyboardInterrupt: 

In [123]:
yolo_pred.keys()

dict_keys(['blanket', 'bolt2', 'traffic', 'ball1', 'leaves', 'pedestrian1', 'singer3', 'helicopter', 'godfather', 'soccer1', 'book', 'tunnel', 'gymnastics1', 'basketball', 'birds1', 'car1', 'ball2', 'birds2', 'motocross1', 'bmx', 'gymnastics2', 'fish3', 'motocross2', 'soccer2', 'octopus', 'fernando', 'shaking', 'crossing', 'singer2', 'soldier', 'pedestrian2', 'nature', 'butterfly', 'road', 'iceskater2', 'sphere', 'fish2', 'graduate', 'marching', 'rabbit', 'car2', 'matrix', 'girl', 'tiger', 'gymnastics4', 'wiper', 'racing', 'dinosaur', 'bolt1', 'hand', 'handball1', 'bag', 'sheep', 'iceskater1', 'fish1', 'singer1', 'gymnastics3', 'fish4', 'glove', 'handball2'])

In [ ]:
for n, vot_class in enumerate(sorted([x for x in os.listdir(vot_path) if not x.endswith(".txt")])):
    print("\n", vot_class, "{}/{}".format(n + 1, len(os.listdir(vot_path))))
    do_dp(vot_class, total_res)

In [32]:
def save(res, folder, ext="csv", nm_fr=None):
    if not os.path.exists(folder):
        os.makedirs(folder)
    for name in res:
        if nm_fr is not None:
            print(nm_fr[name], file=open(osp.join(folder, name + "." + ext), 'w+'))
            print("\n".join([",".join([str(a) for a in x]) for x in res[name]]),
                  file=open(osp.join(folder, name + "." + ext), 'a'))
        else:
            print("\n".join([",".join([str(a) for a in x]) for x in res[name]]),
                  file=open(osp.join(folder, name + "." + ext), 'w+'))


In [33]:
save(total_res, "descriptors")


In [34]:
def iou(cords1, cords2):
    if len(cords2) < 4:
        if (cords2[0] == 1):
            return 1
        return 1
    if (len(cords1) == 8):
        X1, Y1 = cords1[::2], cords1[1::2]
        cords1 = [min(X1), min(Y1), max(X1),  max(Y1)]
    if (len(cords2) == 8):
        X2, Y2 = cords2[::2], cords2[1::2]
        cords2 = [min(X2), min(Y2), max(X2),  max(Y2)]
    xA = max(cords1[0], cords2[0])
    yA = max(cords1[1], cords2[1])
    xB = min(cords1[2], cords2[2])
    yB = min(cords1[3], cords2[3])
    interArea = max(0, xB - xA + 1e-32) * max(0, yB - yA + 1e-32)
    cords1Area = (cords1[2] - cords1[0] + 1e-32) * \
        (cords1[3] - cords1[1] + 1e-32)
    cords2Area = (cords2[2] - cords2[0] + 1e-32) * \
        (cords2[3] - cords2[1] + 1e-32)
    return interArea / float(cords1Area + cords2Area - interArea)

In [35]:
def open_and_cast(file):
    with open(file, 'r') as f:
        pred = [s.split(',') for s in f.read().split("\n")]
        for i in range(len(pred)):
            if pred[i] == [""]:
                pred.pop(i)
                continue
            pred[i] = [float(x) for x in pred[i]]
    return pred

In [36]:
def grade(gt_path, pred_path, method="mean_iou"):
    gt = open_and_cast(gt_path)
    pred = open_and_cast(pred_path)
    grades = [iou(gt[i], pred[i]) for i in range(len(gt))]
    return sum(grades) / len(grades), grades

In [37]:
def grade_vot(vot_folder, res_folder, gt_filename="groundtruth.txt", ext=".csv"):
    vot_folders = sorted(os.listdir(vot_folder)) #["bolt1"]# 
    res = dict()
    for i in range(len(vot_folders)):
        if (vot_folders[i].endswith(".txt")):
            continue
        gt_file = osp.join(vot_folder, vot_folders[i], gt_filename)
        pred_file = osp.join(res_folder, vot_folders[i] + ext)

        # print("++++++++++++")
        # print(gt_file) 
        # print(pred_file)
        # print("++++++++++++")

        single_grade = grade(gt_file, pred_file)[0]
        res[vot_folders[i]] = single_grade
    mean = 0
    for i in res:
        mean += res[i]
    mean /= len(res.keys())
    return mean, res

In [ ]:
grade_vot(vot_path, "descriptors")

In [38]:
params = list()
for i in range(0, 4):
    for j in range(0, 4):
        for k in range(0, 4):
            if not (k == j == i):
                params.append((i,j,k))

In [39]:
params = params[0:10]

In [ ]:
for param in params:
    total_res = dict()
    crit_vals["cc"], crit_vals["iou"], crit_vals["vd"] = param
    dirname = osp.join("param_tests", '_'.join(['%s%s' % (key, value) for (key, value) in crit_vals.items()]))
    
    print(crit_vals)
    start = time.time()
    
    for n, vot_class in enumerate(sorted([x for x in os.listdir(vot_path) if not x.endswith(".txt")])):
        
        print(vot_class, end=" ")
        do_dp(vot_class, total_res)
        
    save(total_res, dirname)
    a = grade_vot(vot_path, dirname)  
    print(a, file = open(dirname + ".txt", "w+"))
        
    done = time.time()
    elapsed = done - start
    print(elapsed)
    
    print()        

{'iou': 0, 'cc': 0, 'id': 1, 'vd': 1}
bag ball1 ball2 basketball birds1 birds2 blanket bmx bolt1 bolt2 book butterfly car1 car2 crossing dinosaur fernando fish1 fish2 fish3 fish4 girl glove godfather graduate gymnastics1 gymnastics2 gymnastics3 gymnastics4 hand handball1 handball2 helicopter iceskater1 iceskater2 leaves marching matrix motocross1 motocross2 nature octopus pedestrian1 pedestrian2 rabbit racing road shaking sheep singer1 singer2 singer3 soccer1 soccer2 soldier sphere tiger traffic tunnel wiper 2659.3934247493744

{'iou': 0, 'cc': 0, 'id': 1, 'vd': 2}
bag ball1 ball2 basketball birds1 birds2 blanket bmx bolt1 bolt2 book butterfly car1 car2 crossing dinosaur fernando fish1 fish2 fish3 fish4 girl glove godfather graduate gymnastics1 gymnastics2 gymnastics3 gymnastics4 hand handball1 handball2 helicopter iceskater1 iceskater2 leaves marching matrix motocross1 motocross2 nature octopus pedestrian1 pedestrian2 rabbit racing road shaking sheep singer1 singer2 singer3 soccer1 so

In [89]:

time.sleep(10)  # or do something more productive



10.010620355606079


In [54]:
import matplotlib.pyplot as plt
from PIL import Image
import os
import os.path as osp
import argparse
import torch
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
from random import randrange
import numpy as np
from matplotlib.pyplot import figure

In [106]:
def plot_single_arr(im_path, pr_arr=None, pr_idx=0, gt_arr=None, gt_idx=0, force_square=False):
    im = Image.open(im_path)
    plt.imshow(im)

    if pr_arr is not None:
        l = pr_arr[pr_idx]
        if l == "":
            return
        pr_bb = [float(x) for x in l.split(',')]
        if len(pr_bb) < 4:
            return
        if force_square:
            X, Y = pr_bb[::2], pr_bb[1::2]
            pr_bb = [min(X), min(Y), max(X),  max(Y)]
            plt.plot([pr_bb[0], pr_bb[2], pr_bb[2], pr_bb[0], pr_bb[0], ],
                     [pr_bb[1], pr_bb[1], pr_bb[3], pr_bb[3], pr_bb[1], ], 'r-', lw=2)
        elif (len(pr_bb) == 8):
            plt.plot(pr_bb[::2] + [pr_bb[0]],
                     pr_bb[1::2] + [pr_bb[1]], 'r-', lw=2)
        else:
            plt.plot([pr_bb[0], pr_bb[2], pr_bb[2], pr_bb[0], pr_bb[0], ],
                     [pr_bb[1], pr_bb[1], pr_bb[3], pr_bb[3], pr_bb[1], ], 'r-', lw=2)

    if gt_arr is not None:
        l = gt_arr[gt_idx]
        if l == "":
            return
        gt_bb = [float(x) for x in l.split(',')]

        if (force_square):
            X, Y = gt_bb[::2], gt_bb[1::2]
            gt_bb = [min(X), min(Y), max(X),  max(Y)]
            plt.plot([gt_bb[0], gt_bb[2], gt_bb[2], gt_bb[0], gt_bb[0], ],
                     [gt_bb[1], gt_bb[1], gt_bb[3], gt_bb[3], gt_bb[1], ], 'b-', lw=2)
        elif (len(gt_bb) == 8):
            plt.plot(gt_bb[::2] + [gt_bb[0]],
                     gt_bb[1::2] + [gt_bb[1]], 'b-', lw=2)
        else:
            plt.plot([gt_bb[0], gt_bb[2], gt_bb[2], gt_bb[0], gt_bb[0], ],
                     [gt_bb[1], gt_bb[1], gt_bb[3], gt_bb[3], gt_bb[1], ], 'b-', lw=2)


def plot_single(im_path, pr_path=None, pr_idx=0, gt_path=None, gt_idx=0, force_square=False):
    """
    coords: [x1, y1, x2, y2, x3, y3, x4, y4]
    """

    im = Image.open(im_path)
    plt.imshow(im)

    if pr_path is not None:
        with open(pr_path, 'r') as f:
            l = f.read().split("\n")[pr_idx]
            if l == "":
                return
            pr_bb = [float(x) for x in l.split(',')]

        if (force_square):
            X, Y = pr_bb[::2], pr_bb[1::2]
            pr_bb = [min(X), min(Y), max(X),  max(Y)]
            plt.plot([pr_bb[0], pr_bb[2], pr_bb[2], pr_bb[0], pr_bb[0], ],
                     [pr_bb[1], pr_bb[1], pr_bb[3], pr_bb[3], pr_bb[1], ], 'r-', lw=2)
        elif (len(pr_bb) == 8):
            plt.plot(pr_bb[::2] + [pr_bb[0]],
                     pr_bb[1::2] + [pr_bb[1]], 'r-', lw=2)
        else:
            plt.plot([pr_bb[0], pr_bb[2], pr_bb[2], pr_bb[0], pr_bb[0], ],
                     [pr_bb[1], pr_bb[1], pr_bb[3], pr_bb[3], pr_bb[1], ], 'r-', lw=2)

    if gt_path is not None:
        with open(gt_path, 'r') as f:
            l = f.read().split("\n")[gt_idx]
            gt_bb = [float(x) for x in l.split(',')]

        if (force_square):
            X, Y = gt_bb[::2], gt_bb[1::2]
            gt_bb = [min(X), min(Y), max(X),  max(Y)]
            plt.plot([gt_bb[0], gt_bb[2], gt_bb[2], gt_bb[0], gt_bb[0], ],
                     [gt_bb[1], gt_bb[1], gt_bb[3], gt_bb[3], gt_bb[1], ], 'b-', lw=2)
        elif (len(gt_bb) == 8):
            plt.plot(gt_bb[::2] + [gt_bb[0]],
                     gt_bb[1::2] + [gt_bb[1]], 'b-', lw=2)
        else:
            plt.plot([gt_bb[0], gt_bb[2], gt_bb[2], gt_bb[0], gt_bb[0], ],
                     [gt_bb[1], gt_bb[1], gt_bb[3], gt_bb[3], gt_bb[1], ], 'b-', lw=2)


def save_plot_single_arr(im_path, name="plot.jpg", pr_arr=None, pr_idx=0, gt_arr=None, gt_idx=0, force_square=False):
    plot_single_arr(im_path, pr_arr, pr_idx, gt_arr, gt_idx, force_square)
    plt.show()
    # plt.savefig(name)


def save_plot_single(im_path, name="plot.jpg", pr_path=None, pr_idx=0, gt_path=None, gt_idx=0, force_square=False):
    plot_single(im_path, pr_path, pr_idx, gt_path, gt_idx, force_square)
    plt.savefig(name)


def save_plot_folder(dir_path, saveto="results", pr_path=None, gt_path=None, force_square=False):
    if (not os.path.exists(saveto)):
        os.makedirs(saveto)
    files = sorted([x for x in os.listdir(dir_path) if x.endswith(".jpg")])

    if gt_path is not None:
        with open(gt_path, 'r') as f:
            gt_arr = f.read().split("\n")

    if pr_path is not None:
        with open(pr_path, 'r') as f:
            pr_arr = f.read().split("\n")

    for n, file in enumerate(files):
        name = osp.join(saveto, file)
#         print("saving {}".format(name))
        save_plot_single_arr(osp.join(dir_path, file), name=name, pr_arr=pr_arr,
                             pr_idx=n, gt_arr=gt_arr, gt_idx=n, force_square=force_square)
        plt.close()


def csv2tensor(path_to_file, first_len=False):
    with open(path_to_file, 'r') as f:
        pr_arr = f.read().split("\n")
        if first_len:
            im_len = int(pr_arr.pop(0))
        else:
            im_len = -1
        pr_arr.pop(-1)
        pr_arr = [[float(y) for y in x.split(",")] for x in pr_arr]
        return torch.tensor(pr_arr), im_len


def plot_single_yolo(im_path, pr_arr, pr_idx, gt_arr, gt_idx, force_square, gs=False):
    im = np.array(Image.open(im_path), dtype=np.uint8)
    # Create figure and axes
    fig, ax = plt.subplots(1)
    # Display the image
    ax.imshow(im)
    # Create a Rectangle patch
    for row in pr_arr[pr_arr[:, 0] == pr_idx]:
        color = (randrange(1, 99) / 100, randrange(1, 99) /
                 100, randrange(1, 99) / 100) if gs == False else (.7, .7, .7)
        label = classes[int(row[-1])]
        x, y = float(row[1]), float(row[2])
        h, w = float(row[4]) - y, float(row[3]) - x
        rect = patches.Rectangle(
            (x, y), w, h, linewidth=2, edgecolor=color, facecolor='none', label=label)
        # Add the patch to the Axes
        ax.add_patch(rect)
        ax.annotate(label, (x, y), color=color, weight='bold',
                    fontsize=10, ha='left', va='top')
    # plt.show()


def save_plot_single_yolo(im_path, name="plot.jpg", pr_tensor=None, pr_idx=0, gt_tensor=None, gt_idx=0, force_square=False, gs=False):
    plot_single_yolo(im_path, pr_tensor, pr_idx,
                     gt_tensor, gt_idx, force_square, gs)
    # plt.savefig(name)


def save_plot_everything(imgs_folder=None, yolo_pred_file=None, pred_file=None, gt_file=None, saveto="results", force_square=False):
    plt.figure(figsize=(60, 80))
    
    if (not os.path.exists(saveto)):
        os.makedirs(saveto)
    files = sorted([x for x in os.listdir(imgs_folder) if x.endswith(".jpg")])

    if gt_file is not None:
        gt_tensor, gt_len = csv2tensor(gt_file)

    if pred_file is not None:
        pred_tensor, pred_len = csv2tensor(pred_file)

    if pred_file is not None:
        yolo_pred_tensor, yolo_pred_len = csv2tensor(yolo_pred_file, True)

    for n, file in enumerate(files):
        name = osp.join(saveto, file)
#         print("saving {}".format(name))
        impath = osp.join(imgs_folder, file)

        plot_single_yolo(impath, yolo_pred_tensor, n,
                         gt_tensor, n, force_square, gs=True)
        plot_single(impath, pred_file, n, gt_file, n, force_square=True)
        plt.savefig(name)
        plt.show()
        plt.close()
        

In [107]:
%matplotlib inline

In [44]:
# save_plot_single("/home/zabulskyy/Datasets/vot2016/leaves/00000100.jpg",
#                  pr_path="/home/zabulskyy/Projects/CTU-Research/results/yolo-blind/leaves.txt",
#                  gt_path="/home/zabulskyy/Datasets/vot2016/leaves/groundtruth.txt",
#                  force_square=True, gt_idx=101, pr_idx=101)
cls = "birds2"
met = "../results/first_results_more_cc_first_init"
yolo_pred = "../yolo_predictions/extended/"

# save_plot_folder(osp.join("/home/zabulskyy/Datasets/vot2016", cls), saveto=osp.join("plots", met, cls),
#                  pr_path=osp.join("notebooks/" + met, cls) + ".csv",
#                  gt_path=osp.join(
#                      "/home/zabulskyy/Datasets/vot2016", cls, "groundtruth.txt"),
#                  force_square=True)

save_plot_everything(imgs_folder=osp.join("/home/zabulskyy/Datasets/vot2016", cls), yolo_pred_file=osp.join(yolo_pred, cls+".csv"),
                    pred_file=osp.join("./" + met, cls) + ".csv", saveto="lololo",
                    force_square=True, gt_file=osp.join(
                     "/home/zabulskyy/Datasets/vot2016", cls, "groundtruth.txt"))


# save_plot_folder("/home/zabulskyy/Datasets/vot2016/leaves", saveto="./plots/yolo-first-smart-smart/leaves",
#                 pr_path="./results/yolo-first-smart-smart/leaves.txt",
#                 gt_path="/home/zabulskyy/Datasets/vot2016/leaves/groundtruth.txt",
#                 force_square=True)

NameError: name 'save_plot_everything' is not defined

In [ ]:
dist(a[0].view([1, -1]), a[1].view([1, -1]))

In [61]:
fuck = {'bag': 0.47341449394132396,
  'ball1': 0.04066745546054344,
  'ball2': 0.01968759353660599,
  'basketball': 0.08591364331679942,
  'birds1': 0.3921305659682844,
  'birds2': 0.45740317332263886,
  'blanket': 0.172603747922678,
  'bmx': 0.6300109110970414,
  'bolt1': 0.2808943469874849,
  'bolt2': 0.5475087886257609,
  'book': 0.1441035796794451,
  'butterfly': 0.2156443579467966,
  'car1': 0.6491787929225329,
  'car2': 0.06424491817679769,
  'crossing': 0.5879241609277033,
  'dinosaur': 0.1553241665037338,
  'fernando': 0.515662651959916,
  'fish1': 0.0100746068440364,
  'fish2': 0.07322816945968362,
  'fish3': 0.4081342253822134,
  'fish4': 0.05187287344261048,
  'girl': 0.4962291888002887,
  'glove': 0.04299043131144828,
  'godfather': 0.020180477733239343,
  'graduate': 0.27383351199299727,
  'gymnastics1': 0.4757011380675518,
  'gymnastics2': 0.43384820471782165,
  'gymnastics3': 0.07143412677907762,
  'gymnastics4': 0.6442061806280363,
  'hand': 0.08472788852124055,
  'handball1': 0.0028024224057576316,
  'handball2': 0.06613589283362757,
  'helicopter': 0.02600585680138968,
  'iceskater1': 0.6477765964970537,
  'iceskater2': 0.578808090320845,
  'leaves': 0.0,
  'marching': 0.21135254870586478,
  'matrix': 0.03930155209327502,
  'motocross1': 0.4409126709782752,
  'motocross2': 0.29600126287717954,
  'nature': 0.05235303942705318,
  'octopus': 0.5819586222778861,
  'pedestrian1': 0.36362883539794993,
  'pedestrian2': 0.739905099710942,
  'rabbit': 0.00021607668653657693,
  'racing': 0.765155737134693,
  'road': 0.5549116377884301,
  'shaking': 0.08212679895837427,
  'sheep': 0.6274904191390392,
  'singer1': 0.1426826696760581,
  'singer2': 0.5789162481757931,
  'singer3': 0.1837715864090872,
  'soccer1': 0.048865984914190544,
  'soccer2': 0.02600265236755819,
  'soldier': 0.15208636096990777,
  'sphere': 0.16788527140509976,
  'tiger': 0.025769608266390145,
  'traffic': 0.4642776719210107,
  'tunnel': 0.4350842322685467,
  'wiper': 0.24880621169005496}

In [62]:
theor = {'blanket': 0.18291710391578664, 'bolt2': 0.5673433659957442, 'traffic': 0.5746887751799356, 'ball1': 0.10681770376159226, 'leaves': 0.0, 'pedestrian1': 0.6485712379493929, 'singer3': 0.1838025259729119, 'helicopter': 0.1579965510567808, 'godfather': 0.15425687802065252, 'soccer1': 0.08186977937811162, 'book': 0.17279531273915943, 'tunnel': 0.4368175031466086, 'gymnastics1': 0.655671257247941, 'basketball': 0.6585782702907222, 'birds1': 0.41405993255774654, 'car1': 0.7267896246948623, 'ball2': 0.01968759353660599, 'birds2': 0.6138320619512763, 'motocross1': 0.46280815041663936, 'bmx': 0.6582125426406513, 'gymnastics2': 0.5813593790121165, 'fish3': 0.41171868354285635, 'motocross2': 0.30778527365118846, 'soccer2': 0.49052709110984866, 'octopus': 0.5884044642442106,'fernando': 0.572051807102179, 'shaking': 0.11367979973417805, 'crossing': 0.6059558512367549, 'singer2': 0.6812222153365551, 'soldier': 0.12212306841040095, 'pedestrian2': 0.7399028766237007, 'nature': 0.5475603649345746, 'butterfly': 0.2175166359839938, 'road': 0.564176788029112, 'iceskater2': 0.6318364015558268, 'sphere': 0.20921765986603758, 'fish2': 0.05880601855436447, 'graduate': 0.6654342338461118, 'marching': 0.26653386183293803, 'rabbit': 0.00021607668653657693, 'car2': 0.1292966751871406, 'matrix': 0.06770082796018426, 'girl': 0.7501918525866975, 'tiger': 0.036646759878251195, 'gymnastics4': 0.6688527570458505, 'wiper': 0.2616305577015176, 'racing': 0.765155737134693, 'dinosaur': 0.29709394329196565, 'bolt1': 0.6448989225335147, 'hand': 0.08545657437107575, 'handball1': 0.2327957603232915, 'bag': 0.4744404017094571, 'sheep': 0.6271279244096969, 'iceskater1': 0.6477351861750958, 'fish1': 0.010541884640825602, 'singer1': 0.612334547639468, 'gymnastics3': 0.387277998339298, 'fish4': 0.04605607976707805, 'glove': 0.06290541098156799, 'handball2': 0.2643525882509363}

In [ ]:
1e-2

In [68]:
opa=0
fuc=0
tot=0
for i in theor:
    diff = theor[i] - fuck[i]
    if fuck[i] / (theor[i]+1e-10) > .8:
        diff = "YEAH"
        fuc+=1
    if theor[i] - fuck[i] < 1e-2:
        diff = "HURRAY"
        opa+=1
        
    print(diff, fuck[i], theor[i], i, sep="\t")
    tot+=1
    
opa/tot
fuc/tot
    

YEAH	0.172603747922678	0.18291710391578664	blanket
YEAH	0.5475087886257609	0.5673433659957442	bolt2
YEAH	0.4642776719210107	0.5746887751799356	traffic
0.06615024830104882	0.04066745546054344	0.10681770376159226	ball1
HURRAY	0.0	0.0	leaves
0.28494240255144293	0.36362883539794993	0.6485712379493929	pedestrian1
HURRAY	0.1837715864090872	0.1838025259729119	singer3
0.13199069425539112	0.02600585680138968	0.1579965510567808	helicopter
0.1340764002874132	0.020180477733239343	0.15425687802065252	godfather
0.03300379446392107	0.048865984914190544	0.08186977937811162	soccer1
YEAH	0.1441035796794451	0.17279531273915943	book
HURRAY	0.4350842322685467	0.4368175031466086	tunnel
0.17997011918038913	0.4757011380675518	0.655671257247941	gymnastics1
0.5726646269739228	0.08591364331679942	0.6585782702907222	basketball
YEAH	0.3921305659682844	0.41405993255774654	birds1
YEAH	0.6491787929225329	0.7267896246948623	car1
HURRAY	0.01968759353660599	0.01968759353660599	ball2
0.1564288886286374	0.4574031733226388

0.5666666666666667

In [64]:
prev_frames = []
for frame in video:
    yp = yolo_predictions(frame)
    do_dp(frame, prev_frames)

NameError: name 'video' is not defined